In [1]:
  from code_analyzer_agent import CodeAnalyzerAgent, create_code_analysis_tools

  # Create agent
  agent = CodeAnalyzerAgent(
      context_start_path="context/v1/start-here.md",
      app_name="cbv",
      pattern="ucc",
      source_code_path="source-code"
  )

  # Register tools
  tools, schemas = create_code_analysis_tools(
      source_code_path="source-code",
      context_dir=agent.context_dir
  )

  for tool_name, tool_func, schema in zip(tools.keys(), tools.values(), schemas):
      agent.register_tool(tool_name, tool_func, schema)

  # Run analysis
  response = agent.run("Analyze the cbv application following the UCC discovery process")

📁 Analyzer logging enabled: /Users/Khaled.Alabsi/projects/llm-agent/analyzer_logs

🔍 Code Analyzer Agent Initialized
   App Name: cbv
   Pattern: ucc
   Source Code: source-code
   Context: context/v1/start-here.md
✓ Registered tool: read_file
✓ Registered tool: search_files
✓ Registered tool: grep_code
✓ Registered tool: write_analysis

🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 
CODE ANALYZER RUN STARTED: 2025-11-11 18:39:31
🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 🤖 

────────────────────────────────────────────────────────────────────────────────
ITERATION 1/10
────────────────────────────────────────────────────────────────────────────────

📤 SENDING TO LLM
Messages: 1
Tools available: 4
📥 LLM Response received

🔧 Tool calls requested: 1
   - search_files

🔧 Executing: search_files
   Args: {
  "pattern": "**/*Controller.java",
  "base_path": "ucc/cbv"
}...
   ✓ Executed successfully

───────────────────────

HTTPError: 400 Client Error: Bad Request for url: http://localhost:1234/v1/chat/completions